In [39]:
import torch
import copy
import imp
import numpy as np
utils = imp.load_source('util', '../utils.py')
flatten_model = utils.flatten_model
copy_module = utils.copy_module

In [40]:
class SimpleNet(torch.nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.layers = torch.nn.Sequential(
#                 torch.nn.Linear(2,2),
#                 torch.nn.ReLU(),
                torch.nn.Linear(2,1)
                )
    def forward(self, x):
        return self.layers(x)

In [31]:
class Hook(object):
    
    def __init__(self, module):
#         self.num = num  # serial number of basic layer
        self.module = module
        self.hookF = self.module.register_forward_hook(self.hook_fnF)
        self.hookB = self.module.register_backward_hook(self.hook_fnB)
        self.pself = copy_module(self.module)

        
    def hook_fnF(self, module, input_, output):
        self.input = copy.deepcopy(input_[0].clone().detach())
        self.input.requires_grad_(True)
        self.output = output
        print('hello')

    def hook_fnB(self, module, input_, R):
#         self.input_grad = input_
#         self.output_grad = output
        assert self.input.grad is None
        assert len(R) == 1, 'R should be tupel of len 1'
#         self.input.requires_grad_(True)
#         import ipdb; ipdb.set_trace()
        R = R[0]
        with torch.enable_grad():
            Z = self.pself(self.input)
            S = R /(Z + (Z==0).float()*np.finfo(np.float32).eps)
            Z.backward(S)
            C = self.input.grad
        self.R = self.input * C
#         import ipdb; ipdb.set_trace()
        return tuple(list(input_)+[1]) #backward hook should return tuple of certain len

    def close(self):
        self.hookF.remove()

In [32]:
class LRP():
    
    def __init__(self, model):
        self.model = copy.deepcopy(model)
        self.model = self.model.eval()
        # put hook to each basic layer
        self.hooks = [Hook(module) for num, module
                      in enumerate(utils.flatten_model(self.model))]
        self.output = None
    #    self.rule = rules.__getattr__(rule) #rule - str name of rule

    def forward(self, input_):
        self.output = self.model(input_)
        return self.output

    def relprop(self, R):
       # global model_depth
       # model_depth = len(self.hooks)
        assert self.output is not None, 'forward pass not performed. do .forward() first'
        self.output.backward(R)
       # for h in self.hooks[::-1]:
       #     r = h.relprop(r, self.rule)
        return self.hooks[0].R

    __call__ = forward

In [33]:
lrp = LRP(SimpleNet())
for l in lrp.model.layers:
    print(l)
    if hasattr(l, 'weight'):
        print(l.weight.data)
    if hasattr(l, 'bias'):
        print(l.bias.data)

Linear(in_features=2, out_features=1, bias=True)
tensor([[-0.2879,  0.4322]])
tensor([0.6702])


In [34]:
inp = torch.tensor([1,4], dtype=torch.float32).view(1,1,-1)
o = lrp(inp)
print('output value: ', o)
R = lrp.relprop(o)
# o.backward(torch.tensor([3], dtype=torch.float32).view(1,1,-1))

hello
output value:  tensor([[[2.1109]]], grad_fn=<AddBackward0>)


RuntimeError: hook 'hook_fnB' has returned an incorrect number of values (got 3, but expected 2)

In [45]:
from torch import nn

In [109]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2,2)
        self.s1 = nn.Sigmoid()
        self.fc2 = nn.Linear(2,2)
        self.s2 = nn.Sigmoid()
        self.fc1.weight = torch.nn.Parameter(torch.Tensor([[0.15,0.2],[0.250,0.30]]))
        self.fc1.bias = torch.nn.Parameter(torch.Tensor([0.35]))
        self.fc2.weight = torch.nn.Parameter(torch.Tensor([[0.4,0.45],[0.5,0.55]]))
        self.fc2.bias = torch.nn.Parameter(torch.Tensor([0.6]))
        
    def forward(self, x):
        x= self.fc1(x)
        x = self.s1(x)
        x= self.fc2(x)
        x = self.s2(x)
        return x

net = Net()
print(net)

Net(
  (fc1): Linear(in_features=2, out_features=2, bias=True)
  (s1): Sigmoid()
  (fc2): Linear(in_features=2, out_features=2, bias=True)
  (s2): Sigmoid()
)


In [136]:
net._modules

OrderedDict([('fc1', Linear(in_features=2, out_features=2, bias=True)),
             ('s1', Sigmoid()),
             ('fc2', Linear(in_features=2, out_features=2, bias=True)),
             ('s2', Sigmoid())])